# Getting Started with Replicate
This notebook shows how to run models on [Replicate](https://replicate.com).

Last updated: 2023-04-24

>[Run a model from Python](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=UXa9IwkeokWH)

>[Setup](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=ep0A2pLDnoWK)

>[Run a model](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=Ax6xbVZOpnaV)

>[Run a model in the background](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=xLvskaxwtswn)

>[Run a model in the background and get a webhook](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=X7ZZHjNrunwr)

>[Compose models into a pipeline](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=xyfeLTILu3ad)

>[Get streaming output from a running model](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=WM47DByLrk5l)

>[Cancel a prediction](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=K7kZwHzLwWoM)

>[Load output files](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=5cuMDPg1xjQZ)

>[Next steps](#updateTitle=true&folderId=18DKj1jLZ00WQAwMRuIYM22n5yGpZzb1r&scrollTo=vz6FASGXsefP)



# Setup

To run this notebook, you’ll need to create a [Replicate](https://replicate.com) account and install the Replicate python client.

In [1]:
# install replicate client
%pip install replicate

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# get a token: https://replicate.com/account
from getpass import getpass
import os

REPLICATE_API_TOKEN = getpass()
os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN

··········


# Run a model
You can run any public model on Replicate from your Python code. The following example runs [stability-ai/stable-diffusion](https://replicate.com/stability-ai/stable-diffusion):

In [5]:
import replicate 

output = replicate.run(
  "stability-ai/stable-diffusion:27b93a2413e7f36cd83da926f3656280b2931564ff050bf9575f1fdf9bcd7478",
  input={"prompt": "a programmer in a dark room with a computer and a cup of coffee"}
)

output

['https://replicate.delivery/pbxt/nxwH1b1dvH6iI1Sk71HzlTq4pw4TiQEc5Qfn0nWedVqzhC2QA/out-0.png']

In [6]:
from IPython.display import Image
Image(url=output[0])

Some models receive images as inputs. To pass a file as an input, use a file handle or URL:

In [7]:
# image = open("mystery.jpg", "rb")
# or...
# image = "https://example.com/mystery.jpg"
#
# for this example, let's use the URL from the previous prediction:
image = output[0]

replicate.run(
  "andreasjansson/blip-2:4b32258c42e9efd4288bb9910bc532a69727f9acd26aa08e175713a0a857a608",
  input={"image": output[0], "prompt": "what's in this picture?"}
)

'a man working on a laptop'

# Run a model in the background
You can start a model and run it in the background:

In [4]:
model = replicate.models.get("stability-ai/stablelm-tuned-alpha-7b")
version = model.versions.get("c49dae362cbaecd2ceabb5bd34fdb68413c4ff775111fea065d259d577757beb")

prediction = replicate.predictions.create(
    version=version,
    input={"prompt":"How do you make ratatouille?"})

In [5]:
prediction

Prediction(id='gm56uclhfne4plwyk7nwlmx5pi', error=None, input={'prompt': 'How do you make ratatouille?'}, logs=None, output=None, status='starting', version=Version(id='c49dae362cbaecd2ceabb5bd34fdb68413c4ff775111fea065d259d577757beb', created_at=datetime.datetime(2023, 4, 20, 23, 7, 42, 348263, tzinfo=datetime.timezone.utc), cog_version='0.7.0-beta17', openapi_schema={'info': {'title': 'Cog', 'version': '0.1.0'}, 'paths': {'/': {'get': {'summary': 'Root', 'responses': {'200': {'content': {'application/json': {'schema': {'title': 'Response Root  Get'}}}, 'description': 'Successful Response'}}, 'operationId': 'root__get'}}, '/shutdown': {'post': {'summary': 'Start Shutdown', 'responses': {'200': {'content': {'application/json': {'schema': {'title': 'Response Start Shutdown Shutdown Post'}}}, 'description': 'Successful Response'}}, 'operationId': 'start_shutdown_shutdown_post'}}, '/predictions': {'post': {'summary': 'Predict', 'responses': {'200': {'content': {'application/json': {'schem

In [6]:
prediction.status

'starting'

In [7]:
dict(prediction).keys()

dict_keys(['id', 'error', 'input', 'logs', 'output', 'status', 'version', 'started_at', 'created_at', 'completed_at'])

In [8]:
prediction.reload()

In [9]:
prediction.status

'succeeded'

In [10]:
''.join(prediction.output)

'There’s quite a lot of debate about how best to prepare this dish - I think there are two main ways to cook it: with a “traditional” method using the ramps or eggplant; then for something quicker and easier, cooking your petals on top can work well too! What we chose here was inspired by my friend Pierre Franey who made his own traditional recipe from scratch...'

# Run a model in the background and get a webhook

You can run a model and get a webhook when it completes, instead of waiting for it to finish. 

If you're working locally, we'd recommend using [ngrok](https://ngrok.com/download) for debugging webhooks. It allows you to tunnel your localhost to a public domain.

In [ ]:
model = replicate.models.get("kvfrans/clipdraw")
version = model.versions.get("5797a99edc939ea0e9242d5e8c9cb3bc7d125b1eac21bda852e5cb79ede2cd9b")
prediction = replicate.predictions.create(
    version=version,
    input={"prompt":"Watercolor painting of an underwater submarine"},
    webhook="https://example.com/your-webhook",
    webhook_events_filter=["completed"]
)

# Compose models into a pipeline
You can run a model and feed the output into another model:

In [ ]:
# laionide = replicate.models.get("afiaka87/laionide-v4").versions.get("b21cbe271e65c1718f2999b038c18b45e21e4fba961181fbfae9342fc53b9e05")
# swinir = replicate.models.get("jingyunliang/swinir").versions.get("660d922d33153019e8c263a3bba265de882e7f4f70396546b6c9c8f9d47a021a")
# image = laionide.predict(prompt="avocado armchair")
# upscaled_image = swinir.predict(image=image)

# Get streaming output from a running model
Some models stream output as the model is running. They will return an iterator, and you can iterate over that output:

In [11]:
iterator = replicate.run(
  "replicate/dolly-v2-12b:ef0e1aefc61f8e096ebe4db6b2bacc297daf2ef6899f0f7e001ec445893500e5",
  input={"prompt": "Who was Dolly the sheep?"},
)
for text in iterator:
      print(text)

Dolly
 was
 a
 sheep
 who
 was
 the
 first
 successfully
 cloned
 mammal




# Cancel a prediction
You can cancel a running prediction:

In [12]:
model = replicate.models.get("cjwbw/damo-text-to-video")
version = model.versions.get("1e205ea73084bd17a0a3b43396e49ba0d6bc2e754e9283b2df49fad2dcf95755")

prediction = replicate.predictions.create(
    version=version,
    input={"prompt":"How do you make ratatouille?"})

In [ ]:
prediction.status

'starting'

In [ ]:
prediction.cancel()

In [ ]:
prediction.reload()

In [ ]:
prediction.status

'canceled'

In [13]:
predictions = replicate.predictions.list()

[p.status for p in predictions[:10]]

['starting', 'succeeded', 'succeeded', 'succeeded', 'succeeded']

# Load output files
Output files are returned as HTTPS URLs. You can load an output file as a buffer:

In [14]:
from urllib.request import urlretrieve

model = replicate.models.get("stability-ai/stable-diffusion")
version = model.versions.get("27b93a2413e7f36cd83da926f3656280b2931564ff050bf9575f1fdf9bcd7478")
out = version.predict(prompt="wavy colorful abstract patterns, cgsociety")
urlretrieve(out[0], "/tmp/out.png")

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/out.png'

# Next steps

- Explore our collection of hosted [models](https://replicate.com/explore)



- Learn about how to integrate with [LangChain](https://python.langchain.com/en/latest/modules/models/llms/integrations/replicate.html)



- Note that you can also run models with the raw HTTP API. Refer to the [HTTP API reference](https://replicate.com/docs/reference/http) for more details.